# 3. Read and inspect Sentinel-1 RTC imagery processed by Microsoft Planetary Computer 

This book will demonstrate how to access radiometrically terrain corrected Sentinel-1 imagery from Microsoft Planetary Computer using `stackstac`. STAC stands for spatio-temporal asset catalog, it is a common framework to describe geospatial information and a way for data providers, developers and users to work and communicate efficiently. You can read more about STAC [here](https://stacspec.org/en) and checkout more useful tutorials for working with STAC data.

## Learning goals

**Xarray and python techniques:** <br>
- Introduction to working with STAC data
- Using `pystac` to query cloud-hosted datasets, observe metadata
- Using `stackstac` to read cloud-hosted data as xarray objects
- Using `xarray` to manipulate and organize Sentinel-1 SAR data
- Performing grouping and reductions on `xarrray` objects
- Visualizing `xarray` objects using `FacetGrid`

**High-level science goals:**<br>
- Querying large cloud-hosted dataset
- Accessing cloud-hosted data stored as COGs (cloud-optimized GeoTIFFs)
- Extracting and organizing metadata

In [ ]:
%xmode minimal
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import planetary_computer
import pystac
import rich.table
import xarray as xr
import stackstac

from IPython.display import Image
from pystac_client import Client

import s1_tools

We will use the `pystac_client` package to interact with and query the Microsoft Planetary Computer Sentinel-1 RTC dataset. In the cell below, we will create an object called `catalog` by calling the `.open()` method of the `Client` class. This is establishing a connection with the hosted data at the url provided. Explore the catalog object. You 

## STAC items

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
catalog

Now we will define some parameters to help us query the data catalog for the specific collection, time range and geographic area of interest. 
The function `points2coords()` just helps us to format coordinates for areas of interest.

In the cell below we specify the time range we're interested in as well as the geographic area of interest.

In [ ]:
time_range = "2021-01-01/2022-08-01"
bbox = [88.214935, 27.92767, 88.302, 28.034]

bbox_coords = s1_tools.points2coords(bbox)
bbox_coords

Now we will search the catalog for entries that match our criteria for collection (Sentinel-1 RTC), bbox (our AOI) and datetime (our specified time range):

In [ ]:
search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range)
items = search.item_collection()
len(items)

We've created a few more instances of `pystac_client` classes. Check out the object types below to better familiarize yourself with the pystac package

In [ ]:
print(type(catalog))
print(type(search))
print(type(items))

You can see that `items` is an instance of the class `ItemCollection`, and we can explore it via the embedded html interface.

In [ ]:
items

To make it easier to work with, we can convert  the `items` object to a dictionary, and from there, convert it to a `geopandas.GeoDataFrame`. You can see that the metadata from within each `item` of the `ItemCollection` object is present in the `GeoDataFrame` but its easier to scan and organize this way. 

In [ ]:
df = gpd.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df.head(5)

In [ ]:
df.columns

Now we want to check out a rendered preview of an individual item from the `ItemCollection` object. We do this by calling the `assets` accessor, and supplying the HREF of the rendered preview key.

In [ ]:
Image(url=items[0].assets["rendered_preview"].href)

We can construct a table with metadata for a single scene (ie. a single element of the list `items`)

In [ ]:
table = rich.table.Table("key", "value")
for k, v in sorted(items[0].properties.items()):
    table.add_row(k, str(v))
table

We can also explore the object metadata outside of the table. Try typing `.assets`, `.links`, `.STAC_extensions` and `.properties` onto the term below. 
You can query the object programmatically for the same metadata stored in the table using dictionary syntax on the `properties` accessor.

In [ ]:
items[0]
# items[0].assets
# items[0].links

In [ ]:
items[0].properties

Now that we'e explored the items that fit our query of the dataset and seen the metadata, let's read the data in using xarray. 

We will now use `dask.distributed` to manage our tasks. Confusingly, we will use the dask.distributed class `Client` to interact with the cluster. 

## Reading data using xarray

## TODO 
if keep this here, explain dask distributed

In [ ]:
from dask.distributed import Client

client = Client(processes=False)
print(client.dashboard_link)

The `client.dashboard_link` points to a dask dashboard for the client we've just instantiated. 

Now that we have queried the data that is available from Microsoft Planetary Computer and inspected the metadata using `pystac`, we will use `stackstac` to read the data into our notebook as an xarray object. Calling `stackstac.stack()` produces a lazy `xarray.DataArray` with dask integration out of a STAC `collection` object. 

In [ ]:
type(items)

In the code cell below, you can see that we pass the object `items`, a `pystac.ItemCollection` to the `stackstac.stack()` method. The wrapper `planetary_computer.sign()` uses Planetary Computer subscription key credentials to access the data. `stackstac` passes the metadata from the STAC collection into the xarray object as coordinates allowing you to further organize and manipulate the object to fit your purposes. `stackstac` can also read the data in according to parameters passed during the `stack()` call. In the code cell below we pass parameters for bounding box and coordinate reference system. To specify the resolution as something other than the resolution at which its stored, pass a `resolution = ` argument. 

In [17]:
da = stackstac.stack(planetary_computer.sign(items), bounds_latlon=bbox, epsg=32645)

In [ ]:
da

### Retrieve source granule ID

It will be useful to have the granule ID for the original SAR acquisition and GRD file used to generate the RTC image. The following code demonstrates retrieving the source granule ID from the STAC metadata and adding it as a coordinate variable to the xarray object  containing the RTC imagery.

In [19]:
stac_item = pystac.read_file(
    "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/S1A_IW_GRDH_1SDV_20210602T120544_20210602T120609_038161_0480FD_rtc"
)

In [20]:
def extract_source_granule_pc(rtc_id):
    base_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc/items/"
    full_url = base_url + str(rtc_id)
    stac_item = pystac.read_file(full_url)
    source_granule = stac_item.links[5].target[-62:]
    return source_granule

In [21]:
granule_ls = [
    extract_source_granule_pc(da.isel(time=t).id.values) for t in range(len(da.time))
]

In [25]:
def make_granule_coord_pc(granule_ls):
    """this fn takes a list of granule IDs, extracts acq date for each granule, organizes this as an array that can be assigned as a coord to an xr obj"""

    acq_date = [pd.to_datetime(granule[17:25]) for granule in granule_ls]

    granule_da = xr.DataArray(
        data=granule_ls,
        dims=["time"],
        coords={"time": acq_date},
        attrs={
            "description": "source granule ID S1 GRD files used to process PC RTC images, extracted from STAC metadata"
        },
        name="granule_id",
    )

    return granule_da

In [ ]:
granule_coord = make_granule_coord_pc(granule_ls)
granule_coord

The `granule_coord` object is a 1-dimensional xarray.DataArray containing the source granule IDs for the original GRD files. The values of the `time` coordinate align with the time coordinate of the data object `pc`, allowing us to use the [`xarray.combine_by_coords()`](https://docs.xarray.dev/en/stable/generated/xarray.combine_by_coords.html) function to merge the two into one object.

First, check the dates between the two coordinates are the same (they should be):

In [ ]:
for element in range(len(list(da.time)))[:5]:
    print(list(da.time.values)[element])
    print(list(granule_coord.time.values)[element])
    print("")
print(len(da.time))
print(len(granule_coord.time))

Now, assign granule id as a **non-dimensional coordinate** of the xarray dataset:

In [28]:
da.coords["granule_id"] = ("time", granule_coord.data)

In [29]:
da = da.compute()

In [25]:
# rename for when we store it to use in the comparison notebook3
# da_pc = da
# da_pc

Great, now we have a data cube of Sentinel-1 RTC backscatter imagery with `x`,`y` and `time` dimensions. Take a look at the coordinates and you can see that there is much more information that we can use to query and filter the dataset. 

Let's do a little bit of looking around. We'll define a function to convert the backscatter pixel values from power to dB scale but we won't use it yet. This transformation applies a logarithmic scale to the data which makes visualization easier but we do not want to run any summary statistics on the dB data as it will be distorted.

What if we only wanted to look at imagery from the `VV` band? 

In [30]:
da_vv = da.sel(band="vv")

We can do the same for `VH`:

In [31]:
da_vh = da.sel(band="vh")

Next, what if we wanted to look only at imagery taken during `ascending` or `descending` passes of the satellite? `band` is a dimensional coordinate, so we could use xarray's `.sel()` method, but orbital direction is a non-dimensional coordinate so we need to approach it a bit differently: 

In [ ]:
da_asc = da.where(da["sat:orbit_state"] == "ascending", drop=True)
da_desc = da.where(da["sat:orbit_state"] == "descending", drop=True)
da_asc

You can see that there are 69 time steps from the Ascending orbital pass and that all of the same dimensions and coordinates still exist, so you can subset for just the `VV` data from the ascending passes, or other variables you may be interested in.

Let's take a look at the two polarizations side-by-side. Below, we'll plot the `VV` and `VH` polarizations from the same date next to each other:

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16, 8))
s1_tools.power_to_db(da_asc.sel(band="vv").isel(time=10)).plot(
    cmap=plt.cm.Greys_r, ax=axs[0]
)
s1_tools.power_to_db(da_asc.sel(band="vh").isel(time=10)).plot(
    cmap=plt.cm.Greys_r, ax=axs[1]
);

It looks like there is some interesting variability between the two images. What if we wanted to see how these differences persist over time?

Let's perform a reduction along the `x` and `y` dimensions so that we can get a better idea of this data over time rather than a snapshot:

In [ ]:
da_asc.sel(band="vv").mean(dim=["x", "y"])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

ax.set_title(
    "Mean backscatter from ASCENDING passes for VH band (red) and VV band (blue) over time"
)
s1_tools.power_to_db(da_asc.sel(band="vv").mean(dim=["x", "y"])).plot(
    ax=ax, linestyle="None", marker="o", color="red"
)
s1_tools.power_to_db(da_asc.sel(band="vh").mean(dim=["x", "y"])).plot(
    ax=ax, linestyle="None", marker="o", color="blue"
);

Interesting! It looks like there is more variability in the VH band than the VV band. Over the year, there is about 4 dB variability in the VV band but over twice as much in the VH band. Chapter 2 of the [SAR handbook](https://gis1.servirglobal.net/TrainingMaterials/SAR/Chp2Content.pdf) contains information about how polarization impacts radar returns. 
The above plots are looking only at the ascending passes. Let's take a look at all of the time steps (ascending + descending). Any effects based on the different viewing geometries of the ascending and descending passes should have been removed during the radiometric terrain correction step.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

ax.set_title("Mean backscatter for VH band (red) and VV band (blue) over time")
s1_tools.power_to_db(da.sel(band="vv").mean(dim=["x", "y"])).plot(
    ax=ax, linestyle="None", marker="o", color="red"
)
s1_tools.power_to_db(da.sel(band="vh").mean(dim=["x", "y"])).plot(
    ax=ax, linestyle="None", marker="o", color="blue"
);

Next, let's take a look at how backscatter values vary seasonally. To do this we will use xarray's [groupby()]() and [.facetgrid]() methods.

In [37]:
seasons_gb = da.groupby(da.time.dt.season).mean()
# add the attrs back to the season groupby object
seasons_gb.attrs = da.attrs

In [ ]:
seasons_gb

Use the seasons groupby object and specify the `season` dimension in the `Facetgrid` call to automatically plot mean backscatter for each season:

In [ ]:
fg_vv = s1_tools.power_to_db(seasons_gb.sel(band="vv")).plot(
    col="season", cmap=plt.cm.Greys_r
);

In [ ]:
fg_vh = s1_tools.power_to_db(seasons_gb.sel(band="vh")).plot(
    col="season", cmap=plt.cm.Greys_r
);

In [42]:
ds_pc = da.to_dataset(dim="band")

In [43]:
ds_pc = ds_pc.drop_dims("band")

In [ ]:
ds_pc = ds_pc.drop("sar:polarizations")

Can i avoid writing this to file/storing it? Maybe instead write quick script to create it in next notebooks  ? 

In [ ]:
ds_pc = ds_pc.drop("raster:bands")

In [ ]:
ds_pc

We will see that we run into some errors trying to write this object to disk:

In [46]:
## First, compute
#ds_pc = ds_pc.compute()
# Try to write to zarr
#ds_pc.to_zarr("../data/tutorial2/s1_pc_cube.zarr", mode="w")

In [ ]:
ds_pc.attrs

The issue seems to be related to one of the `ds_pc` attributes: `RasterSpec`. RasterSpec is an object from the `stackstac` library, which we used to read STAC assets into memory as an Xarray `DataArray` object. It holds a dict-like object that has information about the coordinate reference system, spatial extent and spatial resolution of the object. Let's change it to an object that is JSON serializable so that we can write this object to disk with Zarr without losing any of this metadata.

In [47]:
#rasterspec = ds_pc.attrs["spec"]

#spec_dict = {
#    "epsg": rasterspec.epsg,
#    "bounds": rasterspec.bounds,
#    "resolutions_xy": rasterspec.resolutions_xy,
#}
#ds_pc.attrs.pop("spec")
#ds_pc.attrs["spec_dict"] = spec_dict

Great, now we should be able to write this object:

In [ ]:
#ds_pc.to_zarr("../data/tutorial2/s1_pc_cube.zarr", mode="w")

## Wrap up

This notebook demonstrated how to access cloud-hosted data from Microsoft Planetary Computer, some basic dataset organization and preliminary exploration and visualization. The following notebook will compare the Planetary Computer dataset to the ASF dataset.